In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import pycaret.classification as pc
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data_txt.parquet').compute()

# добавляем target, удаляем nan
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','is_male'])[['is_male','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['is_male'].isna()) & (df['is_male'] != 'NA')]
df['is_male'] = df['is_male'].astype('int8')

display(df.head())
display(df.info())

,user_id,region_name,region_count,city_name,city_count,cpe_manufacturer_name,cpe_type_cd,price,holyday_fraction,morning_fraction,...,belnovosti-by.turbopages.org,ngs24-ru.turbopages.org,forum.mfd.ru,yomed.ru,7info.ru,other_url,other_female_urls_frac,other_male_urls_frac,male_probability_by_urls,is_male
0,0,Москва,1,Москва,1,Samsung,smartphone,2990.0,0.243523,0.119,...,0.0,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0.500491,0
1,1,Москва,3,Москва,6,Xiaomi,smartphone,NaN,0.200573,0.323,...,0.0,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,0.523982,0
2,2,Республика Коми,1,Печора,1,Huawei,smartphone,5915.0,0.299270,0.187,...,0.0,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,0.525968,0
3,3,Воронежская область,1,Воронеж,1,Huawei Device Company Limited,smartphone,13990.0,0.181818,0.178,...,0.0,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,0.519823,1
4,4,Краснодарский край,5,Анапа,9,Huawei,smartphone,12990.0,0.326898,0.372,...,0.0,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,0.510070,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 264326 entries, 0 to 269999
Columns: 3017 entries, user_id to is_male
dtypes: float32(3009), int32(1), int8(3), object(4)
memory usage: 3.0+ GB


None

In [3]:
df = df.drop('price',axis=1)
'''nan_urls_frac = df.isna().sum()[df.isna().sum() > 0]
display(nan_urls_frac.head())

display(df.loc[df.price.isna(),['cpe_manufacturer_name_0',
                                'cpe_manufacturer_name_1',
                                'cpe_manufacturer_name_2',
                                'cpe_manufacturer_name_3',
                                'cpe_manufacturer_name_4',
                                'cpe_manufacturer_name_5']])

df.loc[(df['cpe_manufacturer_name_0'] == 0) &
       (df['cpe_manufacturer_name_1'] == 0) &
       (df['cpe_manufacturer_name_2'] == 0) &
       (df['cpe_manufacturer_name_3'] == 0) &
       (df['cpe_manufacturer_name_4'] == 1) &
       (df['cpe_manufacturer_name_5'] == 0), 'price'].dropna().median()'''

"nan_urls_frac = df.isna().sum()[df.isna().sum() > 0]\ndisplay(nan_urls_frac.head())\n\ndisplay(df.loc[df.price.isna(),['cpe_manufacturer_name_0',\n                                'cpe_manufacturer_name_1',\n                                'cpe_manufacturer_name_2',\n                                'cpe_manufacturer_name_3',\n                                'cpe_manufacturer_name_4',\n                                'cpe_manufacturer_name_5']])\n\ndf.loc[(df['cpe_manufacturer_name_0'] == 0) &\n       (df['cpe_manufacturer_name_1'] == 0) &\n       (df['cpe_manufacturer_name_2'] == 0) &\n       (df['cpe_manufacturer_name_3'] == 0) &\n       (df['cpe_manufacturer_name_4'] == 1) &\n       (df['cpe_manufacturer_name_5'] == 0), 'price'].dropna().median()"

In [3]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['is_male','user_id','price'], axis = 1),df['is_male'], test_size = 0.33, random_state = 42)

Pycaret

In [ ]:
#df = df.drop('user_id',axis=1)
s = pc.setup(df, target = 'is_male', session_id = 123,fold=2)

In [ ]:
best = pc.compare_models(include = ['lightgbm', 'catboost'],fold=2)

Дефолт градиент

In [ ]:
model = GradientBoostingClassifier(random_state=42).fit(x_train,y_train)

In [ ]:
print(f'GINI по полу {2 * roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1:2.3f}')
print(classification_report(y_test, model.predict(x_test)))

Дефолт кетбуст

In [5]:
df.select_dtypes(include=['object']).columns.to_list()

['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_type_cd']

In [6]:
pool_train = Pool(x_train, y_train,
                  cat_features = df.select_dtypes(include=['object']).columns.to_list())

pool_test = Pool(x_test, cat_features = df.select_dtypes(include=['object']).columns.to_list())

model = CatBoostClassifier(random_state=42).fit(pool_train)

Learning rate set to 0.093955
0:	learn: 0.6651145	total: 599ms	remaining: 9m 58s
1:	learn: 0.6404405	total: 986ms	remaining: 8m 12s
2:	learn: 0.6203476	total: 1.42s	remaining: 7m 50s
3:	learn: 0.6029225	total: 1.83s	remaining: 7m 35s
4:	learn: 0.5886351	total: 2.26s	remaining: 7m 29s
5:	learn: 0.5766259	total: 2.67s	remaining: 7m 22s
6:	learn: 0.5664837	total: 3.12s	remaining: 7m 23s
7:	learn: 0.5581443	total: 3.5s	remaining: 7m 14s
8:	learn: 0.5512886	total: 3.93s	remaining: 7m 12s
9:	learn: 0.5455790	total: 4.35s	remaining: 7m 10s
10:	learn: 0.5400836	total: 4.74s	remaining: 7m 6s
11:	learn: 0.5354879	total: 5.13s	remaining: 7m 2s
12:	learn: 0.5315316	total: 5.56s	remaining: 7m 2s
13:	learn: 0.5281873	total: 6.04s	remaining: 7m 5s
14:	learn: 0.5251196	total: 6.5s	remaining: 7m 6s
15:	learn: 0.5224072	total: 6.96s	remaining: 7m 7s
16:	learn: 0.5198305	total: 7.4s	remaining: 7m 8s
17:	learn: 0.5175591	total: 7.94s	remaining: 7m 13s
18:	learn: 0.5153842	total: 8.45s	remaining: 7m 16s
19

In [7]:
print(f'GINI по полу {2 * roc_auc_score(y_test, model.predict_proba(pool_test)[:,1]) - 1:2.3f}')
print(classification_report(y_test, model.predict(pool_test)))

GINI по полу 0.762
              precision    recall  f1-score   support

           0       0.79      0.79      0.79     42609
           1       0.80      0.80      0.80     44619

    accuracy                           0.80     87228
   macro avg       0.80      0.80      0.80     87228
weighted avg       0.80      0.80      0.80     87228



GridSearchCV

In [ ]:
parameters = {'C': np.linspace(0.5, 1.5, 11)}
clf = GridSearchCV(GradientBoostingClassifier(random_state=42), parameters, cv=5, verbose=5)# n_jobs=-1
clf.fit(df.drop('is_male',axis=1), df['is_male'])